# Model Output Analysis: Conversation Topic Classifier

Comprehensive analysis of AI classification outputs including topic labels, operational metadata, and cross-field correlations.

## Notebook Structure

1. **Setup & Data Loading** - Import libraries, load dataset, categorical overview
2. **Gold Set Evaluation** - Compare against manual labels
3. **Topic Distribution Analysis** - Classification patterns
4. **Confidence Analysis** - Model certainty patterns
5. **Emotion Analysis** - Customer emotional state patterns
6. **Difficulty Analysis** - Resolution difficulty patterns
7. **Risk & Escalation Analysis** - Risk levels and escalation triggers
8. **Operational Actions Analysis** - Recommended actions patterns
9. **Root Cause Analysis** - Root cause code patterns
10. **Cross-Field Correlations** - Relationships between categorical fields
11. **Handler Actionability** - Summary quality analysis
12. **Model Health Dashboard** - Summary metrics

---
## 1. Setup & Data Loading

In [ ]:
"""
Import required libraries for data analysis and visualization.
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import ast
from collections import Counter
from itertools import combinations
import warnings

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

# Chart export settings
EXPORT_CHARTS = False
CHART_DPI = 150

print("Libraries loaded successfully.")

In [ ]:
"""
Load the model output dataset.
"""
AI_LABELS_PATH = "data/conversations_ai_classified.csv"
MANUAL_LABELS_PATH = 'data/conversations_ai_classified_gpt5_2_simple.csv'

df = pd.read_csv(AI_LABELS_PATH)

# Parse list columns
def safe_parse_list(val):
    if pd.isna(val):
        return []
    if isinstance(val, list):
        return val
    try:
        return ast.literal_eval(val)
    except:
        return []

df['operational_actions'] = df['operational_actions'].apply(safe_parse_list)
df['escalation_flags'] = df['escalation_flags'].apply(safe_parse_list)

print(f"Loaded {len(df)} conversations")
print(f"\nColumns: {list(df.columns)}")

In [ ]:
"""
Overview of all categorical columns in the dataset.
"""
categorical_cols = ['topic', 'confidence', 'emotion', 'difficulty', 'risk_level', 
                    'escalation_required', 'root_cause_code']

print("="*70)
print("CATEGORICAL COLUMNS OVERVIEW")
print("="*70)

for col in categorical_cols:
    print(f"\n{col.upper()}:")
    print(f"  Unique values: {df[col].nunique()}")
    print(f"  Values: {df[col].unique().tolist()[:10]}")
    
# List columns
print(f"\nOPERATIONAL_ACTIONS (list column):")
all_actions = [a for actions in df['operational_actions'] for a in actions]
print(f"  Unique actions: {len(set(all_actions))}")
print(f"  Total occurrences: {len(all_actions)}")

print(f"\nESCALATION_FLAGS (list column):")
all_flags = [f for flags in df['escalation_flags'] for f in flags]
print(f"  Unique flags: {len(set(all_flags))}")
print(f"  Total occurrences: {len(all_flags)}")

---
## 2. Gold Set Evaluation

Compare AI classifications against manually labeled data.

In [ ]:
"""
Load and merge gold (manual) labels for evaluation.
"""
try:
    manual_df = pd.read_csv(MANUAL_LABELS_PATH)
    pred_df = df[['conversation_id', 'topic']].rename(columns={'topic': 'label_pred'})
    true_df = manual_df[['conversation_id', 'topic']].rename(columns={'topic': 'label_true'})
    eval_df = true_df.merge(pred_df, on='conversation_id', how='inner').dropna()
    GOLD_SET_AVAILABLE = len(eval_df) > 0
    print(f"Gold set loaded: {len(eval_df)} conversations")
except Exception as e:
    GOLD_SET_AVAILABLE = False
    print(f"No gold set available: {e}")

In [ ]:
"""
Classification report comparing AI vs manual labels.
"""
if GOLD_SET_AVAILABLE:
    print("Classification Report (AI vs Manual Labels):")
    print("="*70)
    print(classification_report(eval_df['label_true'], eval_df['label_pred']))
    accuracy = (eval_df['label_true'] == eval_df['label_pred']).mean() * 100
    print(f"\nOverall Accuracy: {accuracy:.1f}%")
else:
    print("Skipped - no gold set available.")

In [ ]:
"""
Confusion matrix visualization.
"""
if GOLD_SET_AVAILABLE and len(eval_df) > 0:
    labels = sorted(set(eval_df['label_true'].unique()) | set(eval_df['label_pred'].unique()))
    cm = confusion_matrix(eval_df['label_true'], eval_df['label_pred'], labels=labels)
    
    fig, ax = plt.subplots(figsize=(12, 10))
    im = ax.imshow(cm, cmap='Blues')
    
    ax.set_xticks(range(len(labels)))
    ax.set_yticks(range(len(labels)))
    ax.set_xticklabels(labels, rotation=45, ha='right', fontsize=8)
    ax.set_yticklabels(labels, fontsize=8)
    
    for i in range(len(labels)):
        for j in range(len(labels)):
            text = ax.text(j, i, cm[i, j], ha='center', va='center', fontsize=8)
    
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('True Label')
    ax.set_title('Confusion Matrix: AI vs Manual Labels', fontsize=14, fontweight='bold')
    plt.colorbar(im)
    plt.tight_layout()
    plt.show()
else:
    print("Confusion matrix skipped.")

---
## 3. Topic Distribution Analysis

In [ ]:
"""
Topic distribution statistics.
"""
topic_counts = df['topic'].value_counts()
topic_pcts = df['topic'].value_counts(normalize=True) * 100

topic_dist = pd.DataFrame({
    'Count': topic_counts,
    'Percentage': topic_pcts.round(1)
})

print(f"Total topics: {len(topic_counts)}")
print(f"\nTopic Distribution:")
topic_dist

In [ ]:
"""
Topic distribution bar chart.
"""
fig, ax = plt.subplots(figsize=(10, 6))

topic_counts_sorted = topic_counts.sort_values(ascending=True)
colors = plt.cm.tab10(np.linspace(0, 1, len(topic_counts)))
bars = ax.barh(topic_counts_sorted.index, topic_counts_sorted.values, color=colors)

for bar, count in zip(bars, topic_counts_sorted.values):
    ax.text(bar.get_width() + 5, bar.get_y() + bar.get_height()/2, 
            f'{count} ({count/len(df)*100:.1f}%)', va='center', fontsize=9)

ax.set_xlabel('Number of Conversations')
ax.set_title('Topic Distribution', fontsize=14, fontweight='bold')
ax.set_xlim(0, max(topic_counts_sorted.values) * 1.25)
plt.tight_layout()
plt.show()

---
## 4. Confidence Analysis

In [ ]:
"""
Overall confidence distribution.
"""
conf_counts = df['confidence'].value_counts().reindex(['high', 'medium', 'low']).fillna(0).astype(int)
conf_pcts = (conf_counts / len(df) * 100).round(1)

print("Overall Confidence Distribution:")
pd.DataFrame({'Count': conf_counts, 'Percentage': conf_pcts})

In [ ]:
"""
Confidence by topic heatmap.
"""
conf_by_topic = pd.crosstab(df['topic'], df['confidence'], normalize='index') * 100
for col in ['high', 'medium', 'low']:
    if col not in conf_by_topic.columns:
        conf_by_topic[col] = 0
conf_by_topic = conf_by_topic[['high', 'medium', 'low']]

fig, ax = plt.subplots(figsize=(8, 8))
im = ax.imshow(conf_by_topic.values, cmap='RdYlGn', aspect='auto', vmin=0, vmax=100)

ax.set_xticks(range(len(conf_by_topic.columns)))
ax.set_yticks(range(len(conf_by_topic.index)))
ax.set_xticklabels(conf_by_topic.columns)
ax.set_yticklabels(conf_by_topic.index, fontsize=8)

for i in range(len(conf_by_topic.index)):
    for j in range(len(conf_by_topic.columns)):
        val = conf_by_topic.iloc[i, j]
        ax.text(j, i, f'{val:.0f}%', ha='center', va='center', fontsize=9)

ax.set_title('Confidence by Topic (%)', fontsize=14, fontweight='bold')
plt.colorbar(im, label='Percentage')
plt.tight_layout()
plt.show()

---
## 5. Emotion Analysis

Analyzing customer emotional states to understand sentiment patterns across topics.

In [ ]:
"""
Overall emotion distribution.
"""
emotion_counts = df['emotion'].value_counts()
emotion_pcts = df['emotion'].value_counts(normalize=True) * 100

print("Emotion Distribution:")
emotion_dist = pd.DataFrame({
    'Count': emotion_counts,
    'Percentage': emotion_pcts.round(1)
})
emotion_dist

In [ ]:
"""
Emotion distribution bar chart with color coding.
"""
emotion_colors = {
    'calm': '#2ecc71',
    'confused': '#f39c12',
    'frustrated': '#e67e22',
    'angry': '#e74c3c',
    'anxious': '#9b59b6',
    'urgent': '#c0392b'
}

fig, ax = plt.subplots(figsize=(10, 5))
emotions = emotion_counts.index.tolist()
colors = [emotion_colors.get(e, '#95a5a6') for e in emotions]

bars = ax.bar(emotions, emotion_counts.values, color=colors)

for bar, count, pct in zip(bars, emotion_counts.values, emotion_pcts.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
            f'{count}\n({pct:.1f}%)', ha='center', fontsize=9)

ax.set_xlabel('Emotion')
ax.set_ylabel('Count')
ax.set_title('Customer Emotion Distribution', fontsize=14, fontweight='bold')
ax.set_ylim(0, max(emotion_counts.values) * 1.2)
plt.tight_layout()
plt.show()

In [ ]:
"""
Emotion by Topic heatmap.
"""
emotion_by_topic = pd.crosstab(df['topic'], df['emotion'], normalize='index') * 100

fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(emotion_by_topic.values, cmap='YlOrRd', aspect='auto')

ax.set_xticks(range(len(emotion_by_topic.columns)))
ax.set_yticks(range(len(emotion_by_topic.index)))
ax.set_xticklabels(emotion_by_topic.columns, rotation=45, ha='right')
ax.set_yticklabels(emotion_by_topic.index, fontsize=8)

for i in range(len(emotion_by_topic.index)):
    for j in range(len(emotion_by_topic.columns)):
        val = emotion_by_topic.iloc[i, j]
        if val > 0:
            ax.text(j, i, f'{val:.0f}', ha='center', va='center', fontsize=8)

ax.set_title('Emotion Distribution by Topic (%)', fontsize=14, fontweight='bold')
plt.colorbar(im, label='Percentage')
plt.tight_layout()
plt.show()

print("\nEmotion by Topic (counts):")
pd.crosstab(df['topic'], df['emotion'])

In [ ]:
"""
Emotion intensity score by topic.
Scale: calm=0, confused=1, anxious=2, frustrated=3, angry=4, urgent=5
"""
emotion_intensity = {
    'calm': 0, 'confused': 1, 'anxious': 2, 
    'frustrated': 3, 'angry': 4, 'urgent': 5
}

df['emotion_score'] = df['emotion'].map(emotion_intensity)

intensity_by_topic = df.groupby('topic')['emotion_score'].agg(['mean', 'std', 'count'])
intensity_by_topic = intensity_by_topic.sort_values('mean', ascending=False)
intensity_by_topic.columns = ['Avg Intensity', 'Std Dev', 'Count']

print("Emotion Intensity by Topic (0=calm to 5=urgent):")
intensity_by_topic.round(2)

In [ ]:
"""
Emotion intensity bar chart.
"""
fig, ax = plt.subplots(figsize=(10, 6))

intensity_sorted = intensity_by_topic['Avg Intensity'].sort_values(ascending=True)
colors = plt.cm.RdYlGn_r(intensity_sorted.values / 5)

bars = ax.barh(intensity_sorted.index, intensity_sorted.values, color=colors)

for bar, val in zip(bars, intensity_sorted.values):
    ax.text(bar.get_width() + 0.05, bar.get_y() + bar.get_height()/2, 
            f'{val:.2f}', va='center', fontsize=9)

ax.set_xlabel('Average Emotion Intensity (0=calm, 5=urgent)')
ax.set_title('Emotion Intensity by Topic', fontsize=14, fontweight='bold')
ax.set_xlim(0, 5.5)
ax.axvline(x=intensity_by_topic['Avg Intensity'].mean(), color='red', linestyle='--', 
           label=f'Avg: {intensity_by_topic["Avg Intensity"].mean():.2f}')
ax.legend()
plt.tight_layout()
plt.show()

---
## 6. Difficulty Analysis

Analyzing resolution difficulty across topics and other dimensions.

In [ ]:
"""
Overall difficulty distribution.
"""
diff_counts = df['difficulty'].value_counts().reindex(['low', 'medium', 'high']).fillna(0).astype(int)
diff_pcts = (diff_counts / len(df) * 100).round(1)

print("Difficulty Distribution:")
pd.DataFrame({'Count': diff_counts, 'Percentage': diff_pcts})

In [ ]:
"""
Difficulty distribution bar chart.
"""
diff_colors = {'low': '#2ecc71', 'medium': '#f39c12', 'high': '#e74c3c'}

fig, ax = plt.subplots(figsize=(8, 5))
bars = ax.bar(diff_counts.index, diff_counts.values, 
              color=[diff_colors.get(d, 'gray') for d in diff_counts.index])

for bar, count, pct in zip(bars, diff_counts.values, diff_pcts.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10, 
            f'{count}\n({pct}%)', ha='center', fontsize=11, fontweight='bold')

ax.set_xlabel('Difficulty Level')
ax.set_ylabel('Count')
ax.set_title('Resolution Difficulty Distribution', fontsize=14, fontweight='bold')
ax.set_ylim(0, max(diff_counts.values) * 1.2)
plt.tight_layout()
plt.show()

In [ ]:
"""
Difficulty by Topic heatmap.
"""
diff_by_topic = pd.crosstab(df['topic'], df['difficulty'], normalize='index') * 100
for col in ['low', 'medium', 'high']:
    if col not in diff_by_topic.columns:
        diff_by_topic[col] = 0
diff_by_topic = diff_by_topic[['low', 'medium', 'high']]

fig, ax = plt.subplots(figsize=(8, 8))
im = ax.imshow(diff_by_topic.values, cmap='RdYlGn_r', aspect='auto', vmin=0, vmax=100)

ax.set_xticks(range(len(diff_by_topic.columns)))
ax.set_yticks(range(len(diff_by_topic.index)))
ax.set_xticklabels(diff_by_topic.columns)
ax.set_yticklabels(diff_by_topic.index, fontsize=8)

for i in range(len(diff_by_topic.index)):
    for j in range(len(diff_by_topic.columns)):
        val = diff_by_topic.iloc[i, j]
        ax.text(j, i, f'{val:.0f}%', ha='center', va='center', fontsize=9)

ax.set_title('Difficulty by Topic (%)', fontsize=14, fontweight='bold')
plt.colorbar(im, label='Percentage')
plt.tight_layout()
plt.show()

print("\nDifficulty by Topic (counts):")
pd.crosstab(df['topic'], df['difficulty'])

In [ ]:
"""
Difficulty by Emotion heatmap - understanding correlation.
"""
diff_by_emotion = pd.crosstab(df['difficulty'], df['emotion'], normalize='index') * 100

fig, ax = plt.subplots(figsize=(10, 4))
im = ax.imshow(diff_by_emotion.values, cmap='YlOrRd', aspect='auto')

ax.set_xticks(range(len(diff_by_emotion.columns)))
ax.set_yticks(range(len(diff_by_emotion.index)))
ax.set_xticklabels(diff_by_emotion.columns, rotation=45, ha='right')
ax.set_yticklabels(diff_by_emotion.index)

for i in range(len(diff_by_emotion.index)):
    for j in range(len(diff_by_emotion.columns)):
        val = diff_by_emotion.iloc[i, j]
        ax.text(j, i, f'{val:.0f}%', ha='center', va='center', fontsize=10)

ax.set_xlabel('Emotion')
ax.set_ylabel('Difficulty')
ax.set_title('Emotion Distribution by Difficulty Level (%)', fontsize=14, fontweight='bold')
plt.colorbar(im, label='Percentage')
plt.tight_layout()
plt.show()

print("\nDifficulty by Emotion (counts):")
pd.crosstab(df['difficulty'], df['emotion'])

---
## 7. Risk & Escalation Analysis

Analyzing risk levels, escalation requirements, and escalation triggers.

In [ ]:
"""
Risk level distribution.
"""
risk_counts = df['risk_level'].value_counts().reindex(['none', 'low', 'medium', 'high']).fillna(0).astype(int)
risk_pcts = (risk_counts / len(df) * 100).round(1)

print("Risk Level Distribution:")
pd.DataFrame({'Count': risk_counts, 'Percentage': risk_pcts})

In [ ]:
"""
Risk level bar chart.
"""
risk_colors = {'none': '#95a5a6', 'low': '#2ecc71', 'medium': '#f39c12', 'high': '#e74c3c'}

fig, ax = plt.subplots(figsize=(8, 5))
bars = ax.bar(risk_counts.index, risk_counts.values, 
              color=[risk_colors.get(r, 'gray') for r in risk_counts.index])

for bar, count, pct in zip(bars, risk_counts.values, risk_pcts.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10, 
            f'{count}\n({pct}%)', ha='center', fontsize=10)

ax.set_xlabel('Risk Level')
ax.set_ylabel('Count')
ax.set_title('Risk Level Distribution', fontsize=14, fontweight='bold')
ax.set_ylim(0, max(risk_counts.values) * 1.2)
plt.tight_layout()
plt.show()

In [ ]:
"""
Risk level by topic heatmap.
"""
risk_by_topic = pd.crosstab(df['topic'], df['risk_level'], normalize='index') * 100
for col in ['none', 'low', 'medium', 'high']:
    if col not in risk_by_topic.columns:
        risk_by_topic[col] = 0
risk_by_topic = risk_by_topic[['none', 'low', 'medium', 'high']]

fig, ax = plt.subplots(figsize=(8, 8))
im = ax.imshow(risk_by_topic.values, cmap='RdYlGn_r', aspect='auto', vmin=0, vmax=100)

ax.set_xticks(range(len(risk_by_topic.columns)))
ax.set_yticks(range(len(risk_by_topic.index)))
ax.set_xticklabels(risk_by_topic.columns)
ax.set_yticklabels(risk_by_topic.index, fontsize=8)

for i in range(len(risk_by_topic.index)):
    for j in range(len(risk_by_topic.columns)):
        val = risk_by_topic.iloc[i, j]
        ax.text(j, i, f'{val:.0f}%', ha='center', va='center', fontsize=9)

ax.set_title('Risk Level by Topic (%)', fontsize=14, fontweight='bold')
plt.colorbar(im, label='Percentage')
plt.tight_layout()
plt.show()

In [ ]:
"""
Risk level by emotion.
"""
risk_by_emotion = pd.crosstab(df['risk_level'], df['emotion'], normalize='index') * 100

fig, ax = plt.subplots(figsize=(10, 4))
im = ax.imshow(risk_by_emotion.values, cmap='YlOrRd', aspect='auto')

ax.set_xticks(range(len(risk_by_emotion.columns)))
ax.set_yticks(range(len(risk_by_emotion.index)))
ax.set_xticklabels(risk_by_emotion.columns, rotation=45, ha='right')
ax.set_yticklabels(risk_by_emotion.index)

for i in range(len(risk_by_emotion.index)):
    for j in range(len(risk_by_emotion.columns)):
        val = risk_by_emotion.iloc[i, j]
        ax.text(j, i, f'{val:.0f}%', ha='center', va='center', fontsize=10)

ax.set_xlabel('Emotion')
ax.set_ylabel('Risk Level')
ax.set_title('Emotion Distribution by Risk Level (%)', fontsize=14, fontweight='bold')
plt.colorbar(im, label='Percentage')
plt.tight_layout()
plt.show()

In [ ]:
"""
Escalation required distribution.
"""
esc_counts = df['escalation_required'].value_counts()
esc_pcts = df['escalation_required'].value_counts(normalize=True) * 100

print("Escalation Required Distribution:")
pd.DataFrame({'Count': esc_counts, 'Percentage': esc_pcts.round(1)})

In [ ]:
"""
Escalation rate by topic.
"""
esc_by_topic = df.groupby('topic')['escalation_required'].agg(['sum', 'count'])
esc_by_topic['rate'] = (esc_by_topic['sum'] / esc_by_topic['count'] * 100).round(1)
esc_by_topic = esc_by_topic.sort_values('rate', ascending=False)
esc_by_topic.columns = ['Escalations', 'Total', 'Rate (%)']

print("Escalation Rate by Topic:")
esc_by_topic

In [ ]:
"""
Escalation rate by emotion.
"""
esc_by_emotion = df.groupby('emotion')['escalation_required'].agg(['sum', 'count'])
esc_by_emotion['rate'] = (esc_by_emotion['sum'] / esc_by_emotion['count'] * 100).round(1)
esc_by_emotion = esc_by_emotion.sort_values('rate', ascending=False)
esc_by_emotion.columns = ['Escalations', 'Total', 'Rate (%)']

print("Escalation Rate by Emotion:")
esc_by_emotion

In [ ]:
"""
Escalation flags distribution.
"""
all_flags = [f for flags in df['escalation_flags'] for f in flags]
flag_counts = Counter(all_flags)

if flag_counts:
    flag_df = pd.DataFrame.from_dict(flag_counts, orient='index', columns=['Count'])
    flag_df = flag_df.sort_values('Count', ascending=False)
    flag_df['Percentage'] = (flag_df['Count'] / len(df) * 100).round(1)
    
    print(f"Escalation Flags Distribution (conversations with any flag: {sum(len(f) > 0 for f in df['escalation_flags'])}):")
    display(flag_df)
    
    # Bar chart
    fig, ax = plt.subplots(figsize=(12, 5))
    bars = ax.barh(flag_df.index, flag_df['Count'].values, color='#e74c3c')
    
    for bar, count in zip(bars, flag_df['Count'].values):
        ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
                str(count), va='center', fontsize=9)
    
    ax.set_xlabel('Count')
    ax.set_title('Escalation Flags Distribution', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("No escalation flags found in the dataset.")

In [ ]:
"""
Escalation flags co-occurrence analysis.
"""
multi_flag_convos = df[df['escalation_flags'].apply(len) > 1]

if len(multi_flag_convos) > 0:
    flag_pairs = []
    for flags in multi_flag_convos['escalation_flags']:
        flag_pairs.extend(combinations(sorted(flags), 2))
    
    if flag_pairs:
        pair_counts = Counter(flag_pairs)
        print(f"Conversations with multiple flags: {len(multi_flag_convos)}")
        print("\nMost common flag combinations:")
        for pair, count in pair_counts.most_common(10):
            print(f"  {pair[0]} + {pair[1]}: {count}")
    else:
        print("No flag combinations found.")
else:
    print("No conversations with multiple escalation flags.")

---
## 8. Operational Actions Analysis

Analyzing recommended operational actions across conversations.

In [ ]:
"""
Operational actions distribution.
"""
all_actions = [a for actions in df['operational_actions'] for a in actions]
action_counts = Counter(all_actions)

action_df = pd.DataFrame.from_dict(action_counts, orient='index', columns=['Count'])
action_df = action_df.sort_values('Count', ascending=False)
action_df['Percentage'] = (action_df['Count'] / len(df) * 100).round(1)

print(f"Total unique actions: {len(action_counts)}")
print(f"Total action occurrences: {len(all_actions)}")
print(f"Avg actions per conversation: {len(all_actions)/len(df):.2f}")
print(f"\nOperational Actions Distribution:")
action_df

In [ ]:
"""
Operational actions bar chart.
"""
fig, ax = plt.subplots(figsize=(12, 8))

action_sorted = action_df.sort_values('Count', ascending=True)
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(action_sorted)))

bars = ax.barh(action_sorted.index, action_sorted['Count'].values, color=colors)

for bar, count in zip(bars, action_sorted['Count'].values):
    ax.text(bar.get_width() + 2, bar.get_y() + bar.get_height()/2, 
            str(count), va='center', fontsize=8)

ax.set_xlabel('Count')
ax.set_title('Operational Actions Distribution', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
"""
Top actions by topic.
"""
action_topic_data = []
for _, row in df.iterrows():
    for action in row['operational_actions']:
        action_topic_data.append({'topic': row['topic'], 'action': action})

action_topic_df = pd.DataFrame(action_topic_data)

if len(action_topic_df) > 0:
    action_by_topic = pd.crosstab(action_topic_df['action'], action_topic_df['topic'])
    
    # Show top 10 actions
    top_actions = action_df.head(10).index.tolist()
    action_by_topic_top = action_by_topic.loc[action_by_topic.index.isin(top_actions)]
    
    fig, ax = plt.subplots(figsize=(14, 8))
    im = ax.imshow(action_by_topic_top.values, cmap='YlGnBu', aspect='auto')
    
    ax.set_xticks(range(len(action_by_topic_top.columns)))
    ax.set_yticks(range(len(action_by_topic_top.index)))
    ax.set_xticklabels(action_by_topic_top.columns, rotation=45, ha='right', fontsize=7)
    ax.set_yticklabels(action_by_topic_top.index, fontsize=8)
    
    for i in range(len(action_by_topic_top.index)):
        for j in range(len(action_by_topic_top.columns)):
            val = action_by_topic_top.iloc[i, j]
            if val > 0:
                ax.text(j, i, str(val), ha='center', va='center', fontsize=7)
    
    ax.set_title('Top 10 Operational Actions by Topic', fontsize=14, fontweight='bold')
    plt.colorbar(im, label='Count')
    plt.tight_layout()
    plt.show()

In [ ]:
"""
Actions by emotion.
"""
action_emotion_data = []
for _, row in df.iterrows():
    for action in row['operational_actions']:
        action_emotion_data.append({'emotion': row['emotion'], 'action': action})

action_emotion_df = pd.DataFrame(action_emotion_data)

if len(action_emotion_df) > 0:
    action_by_emotion = pd.crosstab(action_emotion_df['action'], action_emotion_df['emotion'])
    
    top_actions = action_df.head(10).index.tolist()
    action_by_emotion_top = action_by_emotion.loc[action_by_emotion.index.isin(top_actions)]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    im = ax.imshow(action_by_emotion_top.values, cmap='YlOrRd', aspect='auto')
    
    ax.set_xticks(range(len(action_by_emotion_top.columns)))
    ax.set_yticks(range(len(action_by_emotion_top.index)))
    ax.set_xticklabels(action_by_emotion_top.columns, rotation=45, ha='right')
    ax.set_yticklabels(action_by_emotion_top.index, fontsize=8)
    
    for i in range(len(action_by_emotion_top.index)):
        for j in range(len(action_by_emotion_top.columns)):
            val = action_by_emotion_top.iloc[i, j]
            if val > 0:
                ax.text(j, i, str(val), ha='center', va='center', fontsize=8)
    
    ax.set_title('Top 10 Actions by Emotion', fontsize=14, fontweight='bold')
    plt.colorbar(im, label='Count')
    plt.tight_layout()
    plt.show()

In [ ]:
"""
Actions by difficulty level.
"""
action_diff_data = []
for _, row in df.iterrows():
    for action in row['operational_actions']:
        action_diff_data.append({'difficulty': row['difficulty'], 'action': action})

action_diff_df = pd.DataFrame(action_diff_data)

if len(action_diff_df) > 0:
    action_by_diff = pd.crosstab(action_diff_df['action'], action_diff_df['difficulty'])
    
    # Ensure column order
    for col in ['low', 'medium', 'high']:
        if col not in action_by_diff.columns:
            action_by_diff[col] = 0
    action_by_diff = action_by_diff[['low', 'medium', 'high']]
    
    top_actions = action_df.head(10).index.tolist()
    action_by_diff_top = action_by_diff.loc[action_by_diff.index.isin(top_actions)]
    
    fig, ax = plt.subplots(figsize=(8, 6))
    im = ax.imshow(action_by_diff_top.values, cmap='RdYlGn_r', aspect='auto')
    
    ax.set_xticks(range(len(action_by_diff_top.columns)))
    ax.set_yticks(range(len(action_by_diff_top.index)))
    ax.set_xticklabels(action_by_diff_top.columns)
    ax.set_yticklabels(action_by_diff_top.index, fontsize=8)
    
    for i in range(len(action_by_diff_top.index)):
        for j in range(len(action_by_diff_top.columns)):
            val = action_by_diff_top.iloc[i, j]
            ax.text(j, i, str(val), ha='center', va='center', fontsize=9)
    
    ax.set_title('Top 10 Actions by Difficulty', fontsize=14, fontweight='bold')
    plt.colorbar(im, label='Count')
    plt.tight_layout()
    plt.show()

In [ ]:
"""
Action co-occurrence analysis.
"""
multi_action_convos = df[df['operational_actions'].apply(len) > 1]

print(f"Conversations with multiple actions: {len(multi_action_convos)} ({len(multi_action_convos)/len(df)*100:.1f}%)")

if len(multi_action_convos) > 0:
    action_pairs = []
    for actions in multi_action_convos['operational_actions']:
        action_pairs.extend(combinations(sorted(actions), 2))
    
    if action_pairs:
        pair_counts = Counter(action_pairs)
        print("\nMost common action combinations:")
        for pair, count in pair_counts.most_common(15):
            print(f"  {pair[0]} + {pair[1]}: {count}")

---
## 9. Root Cause Analysis

Analyzing root cause codes across conversations.

In [ ]:
"""
Root cause code distribution.
"""
rc_counts = df['root_cause_code'].value_counts()
rc_pcts = df['root_cause_code'].value_counts(normalize=True) * 100

rc_df = pd.DataFrame({
    'Count': rc_counts,
    'Percentage': rc_pcts.round(1)
})

print(f"Total unique root cause codes: {len(rc_counts)}")
print(f"\nRoot Cause Code Distribution:")
rc_df

In [ ]:
"""
Root cause code bar chart.
"""
fig, ax = plt.subplots(figsize=(12, 8))

rc_sorted = rc_counts.sort_values(ascending=True)
colors = plt.cm.Paired(np.linspace(0, 1, len(rc_sorted)))

bars = ax.barh(rc_sorted.index, rc_sorted.values, color=colors)

for bar, count in zip(bars, rc_sorted.values):
    ax.text(bar.get_width() + 2, bar.get_y() + bar.get_height()/2, 
            f'{count} ({count/len(df)*100:.1f}%)', va='center', fontsize=8)

ax.set_xlabel('Count')
ax.set_title('Root Cause Code Distribution', fontsize=14, fontweight='bold')
ax.set_xlim(0, max(rc_sorted.values) * 1.25)
plt.tight_layout()
plt.show()

In [ ]:
"""
Root cause by topic heatmap.
"""
rc_by_topic = pd.crosstab(df['root_cause_code'], df['topic'])

fig, ax = plt.subplots(figsize=(14, 10))
im = ax.imshow(rc_by_topic.values, cmap='YlGnBu', aspect='auto')

ax.set_xticks(range(len(rc_by_topic.columns)))
ax.set_yticks(range(len(rc_by_topic.index)))
ax.set_xticklabels(rc_by_topic.columns, rotation=45, ha='right', fontsize=7)
ax.set_yticklabels(rc_by_topic.index, fontsize=8)

for i in range(len(rc_by_topic.index)):
    for j in range(len(rc_by_topic.columns)):
        val = rc_by_topic.iloc[i, j]
        if val > 0:
            ax.text(j, i, str(val), ha='center', va='center', fontsize=7)

ax.set_title('Root Cause Codes by Topic', fontsize=14, fontweight='bold')
plt.colorbar(im, label='Count')
plt.tight_layout()
plt.show()

In [ ]:
"""
Root cause by emotion.
"""
rc_by_emotion = pd.crosstab(df['root_cause_code'], df['emotion'])

# Show top 10 root causes
top_rc = rc_counts.head(10).index.tolist()
rc_by_emotion_top = rc_by_emotion.loc[rc_by_emotion.index.isin(top_rc)]

fig, ax = plt.subplots(figsize=(10, 6))
im = ax.imshow(rc_by_emotion_top.values, cmap='YlOrRd', aspect='auto')

ax.set_xticks(range(len(rc_by_emotion_top.columns)))
ax.set_yticks(range(len(rc_by_emotion_top.index)))
ax.set_xticklabels(rc_by_emotion_top.columns, rotation=45, ha='right')
ax.set_yticklabels(rc_by_emotion_top.index, fontsize=8)

for i in range(len(rc_by_emotion_top.index)):
    for j in range(len(rc_by_emotion_top.columns)):
        val = rc_by_emotion_top.iloc[i, j]
        if val > 0:
            ax.text(j, i, str(val), ha='center', va='center', fontsize=8)

ax.set_title('Top 10 Root Causes by Emotion', fontsize=14, fontweight='bold')
plt.colorbar(im, label='Count')
plt.tight_layout()
plt.show()

In [ ]:
"""
Root cause by difficulty.
"""
rc_by_diff = pd.crosstab(df['root_cause_code'], df['difficulty'])

for col in ['low', 'medium', 'high']:
    if col not in rc_by_diff.columns:
        rc_by_diff[col] = 0
rc_by_diff = rc_by_diff[['low', 'medium', 'high']]

fig, ax = plt.subplots(figsize=(8, 10))
im = ax.imshow(rc_by_diff.values, cmap='RdYlGn_r', aspect='auto')

ax.set_xticks(range(len(rc_by_diff.columns)))
ax.set_yticks(range(len(rc_by_diff.index)))
ax.set_xticklabels(rc_by_diff.columns)
ax.set_yticklabels(rc_by_diff.index, fontsize=8)

for i in range(len(rc_by_diff.index)):
    for j in range(len(rc_by_diff.columns)):
        val = rc_by_diff.iloc[i, j]
        ax.text(j, i, str(val), ha='center', va='center', fontsize=8)

ax.set_title('Root Cause Codes by Difficulty', fontsize=14, fontweight='bold')
plt.colorbar(im, label='Count')
plt.tight_layout()
plt.show()

---
## 10. Cross-Field Correlations

Analyzing relationships between multiple categorical fields to identify patterns.

In [ ]:
"""
Emotion x Difficulty -> Escalation Rate heatmap.
Shows how the combination of emotion and difficulty affects escalation probability.
"""
# Create pivot table: escalation rate by emotion and difficulty
emotion_diff_esc = df.pivot_table(
    index='emotion', 
    columns='difficulty', 
    values='escalation_required', 
    aggfunc='mean'
) * 100

# Reorder columns
for col in ['low', 'medium', 'high']:
    if col not in emotion_diff_esc.columns:
        emotion_diff_esc[col] = 0
emotion_diff_esc = emotion_diff_esc[['low', 'medium', 'high']]

fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(emotion_diff_esc.values, cmap='RdYlGn_r', aspect='auto', vmin=0)

ax.set_xticks(range(len(emotion_diff_esc.columns)))
ax.set_yticks(range(len(emotion_diff_esc.index)))
ax.set_xticklabels(emotion_diff_esc.columns)
ax.set_yticklabels(emotion_diff_esc.index)

for i in range(len(emotion_diff_esc.index)):
    for j in range(len(emotion_diff_esc.columns)):
        val = emotion_diff_esc.iloc[i, j]
        if not pd.isna(val):
            ax.text(j, i, f'{val:.0f}%', ha='center', va='center', fontsize=10)

ax.set_xlabel('Difficulty')
ax.set_ylabel('Emotion')
ax.set_title('Escalation Rate by Emotion x Difficulty (%)', fontsize=14, fontweight='bold')
plt.colorbar(im, label='Escalation Rate %')
plt.tight_layout()
plt.show()

print("\nEscalation Rate by Emotion x Difficulty:")
emotion_diff_esc.round(1)

In [ ]:
"""
Emotion x Risk Level distribution.
"""
emotion_risk = pd.crosstab(df['emotion'], df['risk_level'], normalize='index') * 100

for col in ['none', 'low', 'medium', 'high']:
    if col not in emotion_risk.columns:
        emotion_risk[col] = 0
emotion_risk = emotion_risk[['none', 'low', 'medium', 'high']]

fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(emotion_risk.values, cmap='RdYlGn_r', aspect='auto')

ax.set_xticks(range(len(emotion_risk.columns)))
ax.set_yticks(range(len(emotion_risk.index)))
ax.set_xticklabels(emotion_risk.columns)
ax.set_yticklabels(emotion_risk.index)

for i in range(len(emotion_risk.index)):
    for j in range(len(emotion_risk.columns)):
        val = emotion_risk.iloc[i, j]
        ax.text(j, i, f'{val:.0f}%', ha='center', va='center', fontsize=10)

ax.set_xlabel('Risk Level')
ax.set_ylabel('Emotion')
ax.set_title('Risk Level Distribution by Emotion (%)', fontsize=14, fontweight='bold')
plt.colorbar(im, label='Percentage')
plt.tight_layout()
plt.show()

In [ ]:
"""
Difficulty x Risk Level distribution.
"""
diff_risk = pd.crosstab(df['difficulty'], df['risk_level'], normalize='index') * 100

for col in ['none', 'low', 'medium', 'high']:
    if col not in diff_risk.columns:
        diff_risk[col] = 0
diff_risk = diff_risk[['none', 'low', 'medium', 'high']]

# Reorder index
diff_order = ['low', 'medium', 'high']
diff_risk = diff_risk.reindex([d for d in diff_order if d in diff_risk.index])

fig, ax = plt.subplots(figsize=(8, 4))
im = ax.imshow(diff_risk.values, cmap='RdYlGn_r', aspect='auto')

ax.set_xticks(range(len(diff_risk.columns)))
ax.set_yticks(range(len(diff_risk.index)))
ax.set_xticklabels(diff_risk.columns)
ax.set_yticklabels(diff_risk.index)

for i in range(len(diff_risk.index)):
    for j in range(len(diff_risk.columns)):
        val = diff_risk.iloc[i, j]
        ax.text(j, i, f'{val:.0f}%', ha='center', va='center', fontsize=11)

ax.set_xlabel('Risk Level')
ax.set_ylabel('Difficulty')
ax.set_title('Risk Level Distribution by Difficulty (%)', fontsize=14, fontweight='bold')
plt.colorbar(im, label='Percentage')
plt.tight_layout()
plt.show()

In [ ]:
"""
Confidence x Emotion distribution - does emotion affect classification certainty?
"""
conf_emotion = pd.crosstab(df['confidence'], df['emotion'], normalize='columns') * 100

# Reorder index
conf_order = ['high', 'medium', 'low']
conf_emotion = conf_emotion.reindex([c for c in conf_order if c in conf_emotion.index])

fig, ax = plt.subplots(figsize=(10, 4))
im = ax.imshow(conf_emotion.values, cmap='RdYlGn', aspect='auto')

ax.set_xticks(range(len(conf_emotion.columns)))
ax.set_yticks(range(len(conf_emotion.index)))
ax.set_xticklabels(conf_emotion.columns, rotation=45, ha='right')
ax.set_yticklabels(conf_emotion.index)

for i in range(len(conf_emotion.index)):
    for j in range(len(conf_emotion.columns)):
        val = conf_emotion.iloc[i, j]
        ax.text(j, i, f'{val:.0f}%', ha='center', va='center', fontsize=10)

ax.set_xlabel('Emotion')
ax.set_ylabel('Confidence')
ax.set_title('Confidence Distribution by Emotion (%)', fontsize=14, fontweight='bold')
plt.colorbar(im, label='Percentage')
plt.tight_layout()
plt.show()

In [ ]:
"""
Summary statistics for categorical field correlations.
"""
print("="*70)
print("CROSS-FIELD CORRELATION SUMMARY")
print("="*70)

# Escalation rates by various factors
print("\n1. ESCALATION RATES:")
print(f"   Overall: {df['escalation_required'].mean()*100:.1f}%")
print(f"   By emotion (highest): {df.groupby('emotion')['escalation_required'].mean().idxmax()} "
      f"({df.groupby('emotion')['escalation_required'].mean().max()*100:.1f}%)")
print(f"   By difficulty (highest): {df.groupby('difficulty')['escalation_required'].mean().idxmax()} "
      f"({df.groupby('difficulty')['escalation_required'].mean().max()*100:.1f}%)")

# Average emotion intensity
print("\n2. EMOTION INTENSITY:")
print(f"   Overall avg: {df['emotion_score'].mean():.2f}")
print(f"   By difficulty:")
for diff in ['low', 'medium', 'high']:
    subset = df[df['difficulty'] == diff]
    if len(subset) > 0:
        print(f"      {diff}: {subset['emotion_score'].mean():.2f}")

# Risk levels
print("\n3. RISK DISTRIBUTION:")
for risk in ['none', 'low', 'medium', 'high']:
    count = (df['risk_level'] == risk).sum()
    print(f"   {risk}: {count} ({count/len(df)*100:.1f}%)")

# High-risk profile
high_risk = df[df['risk_level'] == 'high']
if len(high_risk) > 0:
    print("\n4. HIGH-RISK PROFILE:")
    print(f"   Most common emotion: {high_risk['emotion'].mode().values[0] if len(high_risk['emotion'].mode()) > 0 else 'N/A'}")
    print(f"   Most common topic: {high_risk['topic'].mode().values[0] if len(high_risk['topic'].mode()) > 0 else 'N/A'}")
    print(f"   Escalation rate: {high_risk['escalation_required'].mean()*100:.1f}%")

---
## 11. Handler Actionability

Analyzing handler summary quality and actionability.

In [ ]:
"""
Handler summary length and quality analysis.
"""
df['summary_length'] = df['handler_summary'].fillna('').str.len()
df['summary_words'] = df['handler_summary'].fillna('').str.split().str.len()

print("Handler Summary Statistics:")
print(f"  Average length (chars): {df['summary_length'].mean():.0f}")
print(f"  Average length (words): {df['summary_words'].mean():.0f}")
print(f"  Min words: {df['summary_words'].min()}")
print(f"  Max words: {df['summary_words'].max()}")
print(f"  Under 35 words (target): {(df['summary_words'] <= 35).sum()} ({(df['summary_words'] <= 35).mean()*100:.1f}%)")

In [ ]:
"""
Handler summary word count distribution.
"""
fig, ax = plt.subplots(figsize=(10, 5))

ax.hist(df['summary_words'], bins=30, color='#3498db', edgecolor='white', alpha=0.7)
ax.axvline(x=35, color='red', linestyle='--', linewidth=2, label='Target max (35 words)')
ax.axvline(x=df['summary_words'].mean(), color='green', linestyle='-', linewidth=2, 
           label=f'Mean ({df["summary_words"].mean():.0f} words)')

ax.set_xlabel('Word Count')
ax.set_ylabel('Frequency')
ax.set_title('Handler Summary Word Count Distribution', fontsize=14, fontweight='bold')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
"""
Actions per conversation distribution.
"""
df['num_actions'] = df['operational_actions'].apply(len)

action_count_dist = df['num_actions'].value_counts().sort_index()

print("Actions per Conversation:")
print(f"  Average: {df['num_actions'].mean():.2f}")
print(f"  Mode: {df['num_actions'].mode().values[0]}")
print(f"  Max: {df['num_actions'].max()}")
print(f"  Zero actions: {(df['num_actions'] == 0).sum()} ({(df['num_actions'] == 0).mean()*100:.1f}%)")
print(f"\nDistribution:")
for n_actions, count in action_count_dist.items():
    print(f"  {n_actions} action(s): {count} ({count/len(df)*100:.1f}%)")

---
## 12. Model Health Dashboard

Summary metrics and health indicators.

In [ ]:
"""
Model Health Dashboard - Summary metrics.
"""
print("="*70)
print("MODEL HEALTH DASHBOARD")
print("="*70)

# Classification quality
print("\n1. CLASSIFICATION QUALITY")
print(f"   Total conversations: {len(df)}")
print(f"   High confidence rate: {(df['confidence'] == 'high').mean()*100:.1f}%")
print(f"   Low confidence rate: {(df['confidence'] == 'low').mean()*100:.1f}%")
print(f"   Catch-all rate: {(df['topic'] == 'General Enquiries & Multi-Intent').mean()*100:.1f}%")

# Operational signals
print("\n2. OPERATIONAL SIGNALS")
print(f"   Escalation rate: {df['escalation_required'].mean()*100:.1f}%")
print(f"   High risk rate: {(df['risk_level'] == 'high').mean()*100:.1f}%")
print(f"   Avg emotion intensity: {df['emotion_score'].mean():.2f} (0-5 scale)")
print(f"   High difficulty rate: {(df['difficulty'] == 'high').mean()*100:.1f}%")

# Handler output quality
print("\n3. HANDLER OUTPUT QUALITY")
print(f"   Avg summary length: {df['summary_words'].mean():.0f} words")
print(f"   Summaries within target: {(df['summary_words'] <= 35).mean()*100:.1f}%")
print(f"   Avg actions per conversation: {df['num_actions'].mean():.2f}")

# Emotion distribution
print("\n4. EMOTION DISTRIBUTION")
for emotion, count in df['emotion'].value_counts().items():
    print(f"   {emotion}: {count} ({count/len(df)*100:.1f}%)")

# Topic distribution (top 5)
print("\n5. TOP 5 TOPICS")
for topic, count in df['topic'].value_counts().head(5).items():
    print(f"   {topic}: {count} ({count/len(df)*100:.1f}%)")

print("\n" + "="*70)

In [ ]:
"""
Health indicator flags.
"""
print("HEALTH INDICATORS:")
print("="*50)

indicators = []

# High confidence check
high_conf_rate = (df['confidence'] == 'high').mean()
if high_conf_rate >= 0.85:
    indicators.append(("[PASS]", f"High confidence rate: {high_conf_rate*100:.1f}% (target: >85%)"))
else:
    indicators.append(("[WARN]", f"High confidence rate: {high_conf_rate*100:.1f}% (target: >85%)"))

# Low confidence check
low_conf_rate = (df['confidence'] == 'low').mean()
if low_conf_rate <= 0.15:
    indicators.append(("[PASS]", f"Low confidence rate: {low_conf_rate*100:.1f}% (target: <15%)"))
else:
    indicators.append(("[WARN]", f"Low confidence rate: {low_conf_rate*100:.1f}% (target: <15%)"))

# Catch-all rate check
catchall_rate = (df['topic'] == 'General Enquiries & Multi-Intent').mean()
if catchall_rate <= 0.20:
    indicators.append(("[PASS]", f"Catch-all rate: {catchall_rate*100:.1f}% (target: <20%)"))
else:
    indicators.append(("[WARN]", f"Catch-all rate: {catchall_rate*100:.1f}% (target: <20%)"))

# Escalation sanity check
esc_rate = df['escalation_required'].mean()
if 0.01 <= esc_rate <= 0.30:
    indicators.append(("[PASS]", f"Escalation rate: {esc_rate*100:.1f}% (reasonable range: 1-30%)"))
else:
    indicators.append(("[INFO]", f"Escalation rate: {esc_rate*100:.1f}% (typical range: 1-30%)"))

# Summary length check
summary_compliance = (df['summary_words'] <= 35).mean()
if summary_compliance >= 0.90:
    indicators.append(("[PASS]", f"Summary length compliance: {summary_compliance*100:.1f}% (target: >90%)"))
else:
    indicators.append(("[WARN]", f"Summary length compliance: {summary_compliance*100:.1f}% (target: >90%)"))

for status, message in indicators:
    print(f"{status} {message}")